### First, load data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dropout
from keras.layers.embeddings import Embedding

import random
#import ndac
import sklearn
%matplotlib inline

Using TensorFlow backend.


In [2]:
# read original data from /gscratch/pfaendtner/cnyambura/NovoNordisk_Capstone/dataframes
data = pd.read_csv('/gscratch/pfaendtner/cnyambura/NovoNordisk_Capstone/dataframes/DF_prest.csv', index_col=0)
data.head()

,prest_id,uniprot_id,conc_cf,aa_seq,nt_seq,aa_len
0,140095,G3V3N0,4.3075,IMTAPSSFEQFKVAMNYLQLYNVPDCLEDIQDADCSSSKCSSSASS...,GACAAGCTTGCGGCCGCAATTATGACAGCTCCCTCCAGTTTTGAGC...,139
1,140099,G3V537,2.9154,TYYAWKHELLGSGTCPALPPREVLGMEELEKLPEEQVAEEELECSA...,GACAAGCTTGCGGCCGCAACCTACTATGCCTGGAAGCATGAGCTGC...,144
2,140225,P12724,1.4877,SLHARPPQFTRAQWFAIQHISLNPPRCTIAMRAINNYRWRCKNQNT...,GACAAGCTTGCGGCCGCATCACTCCATGCCAGACCCCCACAGTTTA...,136
3,140235,H0YH02,6.7224,ARALNESKRVNNGNTAPEDSSPAKKTRRCQRQESKKMPVAGGKANK...,GACAAGCTTGCGGCCGCAGCGAGAGCATTAAATGAAAGCAAAAGAG...,123
4,140309,F5GYC5,3.3848,HRKEPGARLEATRGAARPHKQGTKPMITRPSVSQLGEGKCPSSQHL...,GACAAGCTTGCGGCCGCACATCGGAAAGAGCCTGGGGCAAGGCTGG...,124


In [3]:
#check shape of data
data.shape

(45206, 6)

## Setup nt doc and classify expression

In [4]:
#remove sequences that are not divisile by three
def nt_seq_doc(nt_sequence):
    if 'GACAAGCTTGCGGCCGCA' not in nt_sequence:
        return None
    true_nt = nt_sequence.split('GACAAGCTTGCGGCCGCA')[1]
    if len(true_nt) % 3 != 0:
        return None
    return ' '.join([true_nt[i:i+3] 
                     for i in range(0, len(true_nt), 3)])
# split quantiles
def assign_class(conc):
    if conc <= low_cut:
        return 0
    elif conc >= high_cut:
        return 1
    return

data['nt_seq_doc'] = data['nt_seq'].apply(nt_seq_doc)
data = data[pd.notnull(data['nt_seq_doc'])]

# identify high and low classes by conc_cf quantiles
low_cut = data['conc_cf'].quantile(0.25)
high_cut = data['conc_cf'].quantile(0.75)

data['class'] = data['conc_cf'].apply(assign_class)
data = data[pd.notnull(data['class'])]
# check shape
print('data shape: ', data.shape)

data shape:  (22364, 8)


### Model Training and Data Pre-Processing

In [5]:
#only keep proteins that have <5 PrESTs per protein
low_num_uniprots = data.groupby('uniprot_id').count().aa_seq[data.groupby('uniprot_id').count().aa_seq < 5].index.tolist()
#len(low_num_uniprots)
data_filtered = data[data.uniprot_id.isin(low_num_uniprots)]

In [ ]:
#data_filtered

In [6]:
#X = data['nt_seq_doc']
#y = data['class'].values

# Get the number of prESTs per each uniprot
uniprot_counts = data_filtered.groupby('uniprot_id').count().prest_id

# Add all uniprots with a single prEST to the training set
training_uniprots = uniprot_counts[uniprot_counts == 1].index.tolist()

len(training_uniprots)

6397

In [7]:
# Randomly pick 70% of other uniprots and add them to training set
random.seed(10)
other_uniprots = uniprot_counts[uniprot_counts > 1].index.tolist()

k = int(len(other_uniprots)*0.70)
training_uniprots += random.sample(other_uniprots, k)
len(training_uniprots)

10776

In [8]:
# Add all remaining uniprots to test set
testing_uniprots = list(set(uniprot_counts.index.tolist()) - set(training_uniprots))
len(testing_uniprots)

1877

In [9]:
print('Total number of proteins:', len(data_filtered.uniprot_id.unique()))
print('Number of training proteins:', len(training_uniprots))
print('Number of testing proteins:', len(testing_uniprots))

Total number of proteins: 12653
Number of training proteins: 10776
Number of testing proteins: 1877


In [10]:
# Add all prESTs in training uniprots to training set
nt_train = data_filtered[data_filtered.uniprot_id.isin(training_uniprots)]
nt_train.shape

(17019, 8)

In [11]:
# Repeat for test set
nt_test = data_filtered[data_filtered.uniprot_id.isin(testing_uniprots)]
nt_test.shape

(4556, 8)

In [12]:
# define sequence documents
docs_train = list(nt_train['nt_seq_doc'])
# create the tokenizer
t = Tokenizer()
# fit the tokenizer on the documents
t.fit_on_texts(docs_train)

# integer encode documents
X_train = t.texts_to_sequences(docs_train)
y_train = nt_train['class'].values

# repeat to test set 
docs_test = list(nt_test['nt_seq_doc'])
# fit the tokenizer on the documents
t.fit_on_texts(docs_test)

# integer encode documents
X_test = t.texts_to_sequences(docs_test)
y_test = nt_test['class'].values

# create test-train split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# fix random seed for reproducibility
#np.random.seed(7)

# repeat to test set 
docs = list(data['nt_seq_doc'])
# fit the tokenizer on the documents
t.fit_on_texts(docs)

X = t.texts_to_sequences(docs)

# load the dataset but only keep the top n words, zero the rest
top_words = len(t.word_index) + 1

# truncate and pad input sequences
seq_lengths = [len(seq) for seq in X]
max_seq_length = max(seq_lengths)
X_train = sequence.pad_sequences(X_train, maxlen=max_seq_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_seq_length)
#X = sequence.pad_sequences(X, maxlen=max_seq_length)

In [ ]:
# create the model using parameters from grid search 
embedding_vecor_length = 16
drop = 0.5
recurrent_drop = 0.5
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_seq_length))
model.add(Conv1D(filters=200, kernel_size=5, padding='same', activation='selu'))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(150, dropout=drop, recurrent_dropout=recurrent_drop))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# record training progress
history = model.fit(X_train, y_train, epochs=35, batch_size=64, validation_data=(X_test, y_test))

# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Train on 17019 samples, validate on 4556 samples
Epoch 1/35
17019/17019 [==============================] - 20s 1ms/step - loss: 0.6813 - acc: 0.5500 - val_loss: 0.6645 - val_acc: 0.5942

In [ ]:
history.history

In [ ]:
# plot loss vs. epoch
# https://machinelearningmastery.com/diagnose-overfitting-underfitting-lstm-models/
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
y_pred = model.predict(X_test)
len(y_pred)

In [ ]:
nt_test.loc[:,'act_prob'] = y_pred

In [ ]:
nt_test_final = nt_test[['prest_id','uniprot_id','conc_cf','aa_seq','nt_seq','class','act_prob']].sort_values('uniprot_id',ascending=0).reset_index(drop=True)

In [ ]:
print('Original total number of experiments:',len(nt_test_final))
print('Original total number of proteins:',len(nt_test_final.uniprot_id.unique()))

In [ ]:
print('Original Number of passed experiments:',len(nt_test_final[nt_test_final['class']==1]))
print('Original Pass rate: %.2f%%'%(np.true_divide(len(nt_test_final[nt_test_final['class']==1]),len(nt_test))*100))
print('Number of proteins with >1 high expression:',len(nt_test_final[nt_test_final['class']==1].uniprot_id.unique()))

In [ ]:
len(nt_test_final[nt_test_final['class']==1])

In [ ]:
print('True positive:', len(nt_test_final[(nt_test_final['act_prob'] > 0.5)]) & len(nt_test_final[nt_test_final['class']==1]))

In [ ]:
len(nt_test_final[nt_test_final['class']==0])

In [ ]:
print('False positive:', len(nt_test_final[(nt_test_final['act_prob'] > 0.5)]) & len(nt_test_final[nt_test_final['class']==0]))
#print 'False Positive:',len(DF_retro_final[(DF_retro_final.ens_score > .5) & (DF_retro_final.expressed==False)])

In [ ]:
print('True negative:', len(nt_test_final[(nt_test_final['act_prob'] < 0.5)]) & len(nt_test_final[nt_test_final['class']==0]))

In [ ]:
len(nt_test_final[(nt_test_final['act_prob'] < 0.5)])

In [ ]:
print('False negative:', len(nt_test_final[(nt_test_final['act_prob'] < 0.5)]) & len(nt_test_final[nt_test_final['class']==1]))
#print 'False Negative:',len(DF_retro_final[(DF_retro_final.ens_score < .5) & (DF_retro_final.expressed)])

In [ ]:
#grab only the top expressing proteins
n = 5
np.random.seed(0)

output_df = pd.DataFrame(columns=['prest_id','uniprot_id','class','act_prob','nt_seq'])
remaining_df = nt_test_final.copy()

for i in range(n):
    print('Iteration',i)
    new_output_df = remaining_df.sort_values(['uniprot_id','act_prob'],ascending=[1,0]).drop_duplicates('uniprot_id')
    output_df = pd.concat([output_df,new_output_df])
   
    pred_pos_proteins = set(output_df[output_df['act_prob'] > 0.5].uniprot_id)
    true_pos_proteins = set(output_df[output_df['class']==1].uniprot_id)
    print('Total number of proposed experiments:',len(output_df))
    print('Total number of expressed proteins:',len(true_pos_proteins))
    print('Overall pass rate:',np.true_divide(len(true_pos_proteins),len(output_df)))

    # Prepare for next iteration
    remaining_df = remaining_df.drop(new_output_df.index)
    remaining_df = remaining_df[remaining_df.uniprot_id.isin(true_pos_proteins)==False]

    print
print('Percent saved experiments:',(1 - np.true_divide(len(output_df),len(nt_test_final)))*100,'%')

In [ ]:
remain_df = nt_test_final.copy()

In [ ]:
n = 5
np.random.seed(0)
n_output_df = remain_df.sort_values(['uniprot_id','act_prob'],ascending=[1,0]).drop_duplicates('uniprot_id')
oput_df = pd.concat([oput_df,n_output_df])
   
pred_pos_proteins = set(oput_df[oput_df['act_prob'] > 0.5].uniprot_id)
true_pos_proteins = set(oput_df[oput_df['class']==1].uniprot_id)
print('Total number of proposed experiments:',len(oput_df))
print('Total number of expressed proteins:',len(true_pos_proteins))
print('Overall pass rate:',np.true_divide(len(true_pos_proteins),len(oput_df)))

# Prepare for next iteration
remaining_df = remain_df.drop(new_output_df.index)
remaining_df = remain_df[remain_df.uniprot_id.isin(true_pos_proteins)==False]


In [ ]:
remaining_df